##### Calling API with passing parameters

In [ ]:
import requests

# List of coordinates
locations = [
    {"lat": 51.50, "lon": -0.09},
    {"lat": 48.85, "lon": 2.34},
    {"lat": 28.55, "lon": 77.16},
    {"lat": 19.13, "lon": 72.89},
    {"lat": -33.86, "lon": 151.20},
    {"lat": 13.08, "lon": 80.16},
    {"lat": 12.97, "lon": 77.60}
]

api_key = "ada2b9b34c35b27ad11a7c8959ad9f6b"
all_data = []

for loc in locations:
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={loc['lat']}&lon={loc['lon']}&appid={api_key}"
    response = requests.get(url)
    data = response.json()
    all_data.append(data)  # Collect each response
    print(f"Weather at ({loc['lat']}, {loc['lon']}): {data['weather'][0]['description']}")

In [ ]:
# Check if the response is JSON
if 'application/json' in response.headers.get('Content-Type', ''):
    data = response.json()
    print(data)
else:
    print("Received non-JSON content:", response.text[:500])

##### No. of columns with column name

In [7]:
print(df.columns.tolist())
print(f"Total columns: {len(df.columns)}")

##### Using pandas lib, json data will be convert into table : DataFrame
1. Flatten the JSON - This will flatten the nested structure and give you a DataFrame with readable columns.

In [6]:
import pandas as pd
from datetime import datetime
from pandas import json_normalize


# Normalize all responses into a DataFrame
df = json_normalize(all_data)
  # Now you'll see multiple rows! & view the table


# Preserve raw values
df['Sunrise_raw'] = df['sys.sunrise']
df['Sunset_raw']  = df['sys.sunset']

# Convert to UTC datetime in clean format
df['Sunrise_UTC'] = pd.to_datetime(df['Sunrise_raw'], unit='s', utc=True)
df['Sunrise_UTC'] = df['Sunrise_UTC'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['Sunset_UTC']  = pd.to_datetime(df['Sunset_raw'], unit='s', utc=True)
df['Sunset_UTC'] = df['Sunset_UTC'].dt.strftime('%Y-%m-%d %H:%M:%S')


df['dt_raw']=df['dt']
# Extract raw timestamp
dt_raw = data.get("dt")

# Convert to UTC datetime string
dt_utc = datetime.utcfromtimestamp(dt_raw).strftime('%Y-%m-%d %H:%M:%S') if dt_raw else None

#remove unwanted raw columns now
df.drop(columns=['Sunset_raw', 'Sunrise_raw', 'dt_raw','sys.sunrise','sys.sunset','dt'], inplace=True)

df.head()

##### Extract Specific Fields in a cleaner table with key metrics. Collect All Weather Rows

In [8]:
import requests
import pandas as pd
from datetime import datetime


locations = [
    {"lat": 51.50, "lon": -0.09},
    {"lat": 48.85, "lon": 2.34},
    {"lat": 28.55, "lon": 77.16},
    {"lat": 19.13, "lon": 72.89},
    {"lat": -33.86, "lon": 151.20},
    {"lat": 13.08, "lon": 80.16},
    {"lat": 12.97, "lon": 77.60}
            ]


api_key = "ada2b9b34c35b27ad11a7c8959ad9f6b"
weather_rows = []


for loc in locations:
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={loc['lat']}&lon={loc['lon']}&appid={api_key}"
    response = requests.get(url)
    data = response.json()

    dt_utc = datetime.utcfromtimestamp(data.get("dt")).strftime('%Y-%m-%d %H:%M:%S') if data.get("dt") else None
#Row dictionary or ingestion logic
    row = {
           # Coordinates
        "Longitude": data.get("coord", {}).get("lon"),
        "Latitude": data.get("coord", {}).get("lat"),

        # Weather
        "WeatherID": data.get("weather", [{}])[0].get("id"),
        "WeatherMain": data.get("weather", [{}])[0].get("main"),
        "Description": data.get("weather", [{}])[0].get("description"),
        "WeatherIcon": data.get("weather", [{}])[0].get("icon"),

        # Base
        "Base": data.get("base"),

        # Main Metrics
        "Temperature": data.get("main", {}).get("temp"),
        "FeelsLike": data.get("main", {}).get("feels_like"),
        "Pressure (hPa)": data.get("main", {}).get("pressure"),
        "Humidity": data.get("main", {}).get("humidity"),
        "MinTemperature": data.get("main", {}).get("temp_min"),
        "MaxTemperature": data.get("main", {}).get("temp_max"),
        "Pressure SeaLevel": data.get("main", {}).get("sea_level"),
        "Pressure GroundLevel": data.get("main", {}).get("grnd_level"),

        # Visibility
        "Visibility": data.get("visibility"),

        # Wind
        "WindSpeed": data.get("wind", {}).get("speed"),
        "WindDirection": data.get("wind", {}).get("deg"),
        "WindGust": data.get("wind", {}).get("gust"),

        # Clouds
        "Cloudiness": data.get("clouds", {}).get("all"),

        # Rain & Snow
        "Rain (1h mm)": data.get("rain", {}).get("1h"),
        "Snow (1h mm)": data.get("snow", {}).get("1h"),

        # System Info
        "Country": data.get("sys", {}).get("country"),
        #"Sunrise (UTC)": data.get("sys", {}).get("sunrise"),
        #"Sunset (UTC)": data.get("sys", {}).get("sunset"),
        
        # Raw values
        #"Sunrise_raw": data.get("sys", {}).get("sunrise"),
        #"Sunset_raw": data.get("sys", {}).get("sunset"),
    
    # Converted UTC datetime
    "Sunrise": datetime.utcfromtimestamp(data.get("sys", {}).get("sunrise")).strftime('%Y-%m-%d %H:%M:%S') if data.get("sys", {}).get("sunrise") else None,
    "Sunset": datetime.utcfromtimestamp(data.get("sys", {}).get("sunset")).strftime('%Y-%m-%d %H:%M:%S') if data.get("sys", {}).get("sunset") else None,

        "Sys Type": data.get("sys", {}).get("type"),
        "Sys ID": data.get("sys", {}).get("id"),
        "Sys Message": data.get("sys", {}).get("message"),

        

        # Metadata
        "CityName": data.get("name"),
        "CityID": data.get("id"),
        "TimeZone": data.get("timezone"),
        "Response Code": data.get("cod"),
                   
        # Raw and converted timestamp
        "Data Timestamp (UTC)": dt_utc,
        "WeatherDay" : "Current"
        
    }
    weather_rows.append(row)
# Convert list of rows to DataFrame
weather_table = pd.DataFrame(weather_rows)
weather_table

##### Return the number of rows in this data

In [5]:
print(f"Total successful rows: {len(weather_rows)}")
print(weather_table)

##### Exporting this DataFrame to CSV

In [2]:
weather_table.to_csv("weather_data.csv", index=False)

StatementMeta(, 0a2b9df0-4431-4790-910e-964f79761b16, 7, Finished, Available, Finished)

##### Save csv file to Lakehouse Table

In [3]:
#Convert Pandas DataFrame to Spark DataFrame
import pandas as pd
from pyspark.sql import SparkSession

# Your existing Pandas DataFrame
pdf = weather_table

# Convert to Spark DataFrame
spark = SparkSession.builder.getOrCreate()
df = spark.createDataFrame(pdf)


StatementMeta(, 0a2b9df0-4431-4790-910e-964f79761b16, 8, Finished, Available, Finished)

##### Replaced unsupported signs from table columns

In [4]:
weather_table.columns = [col.replace("(", "").replace(")", "").replace(" ", "_") for col in weather_table.columns]

StatementMeta(, 0a2b9df0-4431-4790-910e-964f79761b16, 9, Finished, Available, Finished)

##### Saving file & Loading table into existing lakehouse

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.createDataFrame(weather_table)
df.write.format("delta").mode("overwrite").saveAsTable("Gold_WeatherData_Current")
#df.write.mode("overwrite").saveAsTable("Gold_WeatherData_Current")

StatementMeta(, 0a2b9df0-4431-4790-910e-964f79761b16, 10, Finished, Available, Finished)

##### show tables

In [ ]:
spark.sql("SHOW TABLES").show()


##### Loading table into existing lakehouse

In [ ]:
#df.write.mode("overwrite").saveAsTable("Gold_WeatherData_Current")
#df.write.format("delta").mode("overwrite").save("weather_data")